In [44]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler 
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [45]:
#Nearshore wave
df_wave=pd.read_csv('/home/mandana/Data/NSW Nearshore wave/combined-nsw-wave-data_daily_merged.csv')
df_wave['datetime']= pd.to_datetime(df_wave['datetime'])
df_wave.rename(columns={'datetime':'time',}, inplace=True)
df_wave['beach_key_lower'] = df_wave['beach_key'].str.lower()
df_wave

,date_x,inshore_location_id,beach_key,hst,tpt,dpt,Inshore_Latitude,Inshore_longitude,time,Last_date,beach,distance,date_y,beach_key_lower
0,1994-01-23,104168,nsw450,0.672917,8.822500,126.137500,-35.181713,150.60603,1994-01-23,2018-12-22,Sussex Inlet,0.681801,1994-01-23,nsw450
1,1994-01-24,104168,nsw450,0.486250,7.384583,120.707500,-35.181713,150.60603,1994-01-23,2018-12-22,Sussex Inlet,0.681801,1994-01-23,nsw450
2,1994-01-25,104168,nsw450,0.314583,5.401667,113.441667,-35.181713,150.60603,1994-01-23,2018-12-22,Sussex Inlet,0.681801,1994-01-23,nsw450
3,1994-01-26,104168,nsw450,0.214583,4.686250,110.875417,-35.181713,150.60603,1994-01-23,2018-12-22,Sussex Inlet,0.681801,1994-01-23,nsw450
4,1994-01-27,104168,nsw450,0.766667,5.637083,131.411667,-35.181713,150.60603,1994-01-23,2018-12-22,Sussex Inlet,0.681801,1994-01-23,nsw450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724793,2023-12-31,104677,nsw566,0.777500,8.391667,137.925000,-35.917225,150.15964,2019-12-26,2024-01-27,Moruya (Heads) Beach,0.671339,2019-12-26,nsw566
724794,2023-12-31,104825,nsw599,0.792917,8.709167,136.737500,-36.223408,150.14479,2019-11-02,2024-03-09,Narooma Beach,0.472900,2019-11-02,nsw599
724795,2023-12-31,105072,nsw659,0.658333,8.582500,132.070417,-36.716854,149.98341,2005-12-12,2024-03-09,Tathra Beach,0.547626,2005-12-12,nsw659
724796,2023-12-31,105189,nsw671B,0.365833,8.592083,110.043333,-36.929382,149.91034,2020-11-28,2024-03-10,Pambula Beach,0.627775,2020-11-28,nsw671b


In [46]:
df_coastal=pd.read_csv('/home/mandana/Data/slsa-surfguard-NEW_MG3_final_Bin.csv')
df_coastal['time'] = pd.to_datetime(df_coastal['date'])

In [47]:
df=pd.read_csv('/home/mandana/Data/slsa-surfguard-NEW_MG3_final1_wave_wind_current_rename_cgp.csv')
df['time'] = pd.to_datetime(df['time'])

df=pd.merge(df, df_coastal[['embaymentisation', 'orientation','time', 'beach_key_lower']], on=['beach_key_lower','time'], how='left',)

df['stings_binary'] = 0
df.loc[df['stings'] != 0, 'stings_binary'] = 1


df['day_of_year'] = df['time'].dt.dayofyear 
df['is_weekend'] = df['time'].dt.weekday >= 5  
df['year']=df['time'].dt.year
df = df[df['time'].dt.month.isin([1])].copy() # Filter for all months

df = df.sort_values(['beach_key_lower', 'time'])

# df = df[df['state'] == 'QLD'].copy()
# df = df[df['stings']!=1].copy()  # Exclude rows where stings is 1
# df = df[df['species'] == 'Physalia (Bluebottle)'].copy()



In [ ]:
# # df=df[(df['lat'] >= -28.18) & (df['lat'] <= -27.90) &
# #          (df['lon'] >= 153.40) & (df['lon'] <= 153.58)].copy()

# df=df[(df['lat'] >= -26.88) & (df['lat'] <= -26.3) &
#          (df['lon'] >= 153) & (df['lon'] <= 153.2)].copy()
# df=df[~df['beach_key_lower'].isin(['qld1532','qld1543a','qld1544b'])].copy()

In [48]:
df_combined=pd.merge(df,df_wave,  on=['time','beach_key_lower'], how='inner')

In [49]:
df_combined

,time,lat,lon,species,beach_key_lower,stings,attendance,duration,species.1,state,...,beach_key,hst,tpt,dpt,Inshore_Latitude,Inshore_longitude,Last_date,beach,distance,date_y
0,2010-01-02,-28.1683,153.5514,Physalia (Bluebottle),nsw001,31.0,390,9.0,Physalia (Bluebottle),NSW,...,nsw001,1.295417,7.107500,50.312500,-28.164795,153.55801,2024-02-04,Flagstaff/Duranbah Beach,0.756140,2010-01-02
1,2010-01-02,-28.1683,153.5514,Physalia (Bluebottle),nsw001,31.0,390,9.0,Physalia (Bluebottle),NSW,...,nsw001,1.066250,6.297917,95.557500,-28.164795,153.55801,2024-02-04,Flagstaff/Duranbah Beach,0.756140,2010-01-02
2,2010-01-02,-28.1683,153.5514,Physalia (Bluebottle),nsw001,31.0,390,9.0,Physalia (Bluebottle),NSW,...,nsw001,1.435000,6.957083,107.436250,-28.164795,153.55801,2024-02-04,Flagstaff/Duranbah Beach,0.756140,2010-01-02
3,2010-01-02,-28.1683,153.5514,Physalia (Bluebottle),nsw001,31.0,390,9.0,Physalia (Bluebottle),NSW,...,nsw001,1.202083,6.942500,100.396250,-28.164795,153.55801,2024-02-04,Flagstaff/Duranbah Beach,0.756140,2010-01-02
4,2010-01-02,-28.1683,153.5514,Physalia (Bluebottle),nsw001,31.0,390,9.0,Physalia (Bluebottle),NSW,...,nsw001,0.950000,7.211667,101.213750,-28.164795,153.55801,2024-02-04,Flagstaff/Duranbah Beach,0.756140,2010-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37490,2009-01-24,-28.1568,153.5068,Physalia (Bluebottle),qld1597b,5.0,125,4.5,Physalia (Bluebottle),QLD,...,qld1597B,1.106250,7.635000,81.282500,-28.159600,153.54988,2016-12-26,Greenmount,1.605164,2009-01-24
37491,2009-01-24,-28.1568,153.5068,Physalia (Bluebottle),qld1597b,5.0,125,4.5,Physalia (Bluebottle),QLD,...,qld1597B,1.172500,7.751667,80.506667,-28.159600,153.54988,2016-12-26,Greenmount,1.605164,2009-01-24
37492,2009-01-24,-28.1568,153.5068,Physalia (Bluebottle),qld1597b,5.0,125,4.5,Physalia (Bluebottle),QLD,...,qld1597B,1.801667,9.065833,73.034583,-28.159600,153.54988,2016-12-26,Greenmount,1.605164,2009-01-24
37493,2009-01-24,-28.1568,153.5068,Physalia (Bluebottle),qld1597b,5.0,125,4.5,Physalia (Bluebottle),QLD,...,qld1597B,1.984167,9.879583,72.996667,-28.159600,153.54988,2016-12-26,Greenmount,1.605164,2009-01-24


In [50]:
df_combined.columns

Index(['time', 'lat', 'lon', 'species', 'beach_key_lower', 'stings',
       'attendance', 'duration', 'species.1', 'state', 'U10_wave', 'V10_wave',
       'hs_wave', 't_wave', 'CgE_wave', 'fp_wave', 'sin_dir_wave',
       'cos_dir_wave', 'sfcWind_wind', 'ts_wind', 'uas_wind', 'vas_wind',
       'salt_crt', 'temp', 'u_crt', 'v_crt', 'stings_binary',
       'embaymentisation', 'orientation', 'day_of_year', 'is_weekend', 'year',
       'date_x', 'inshore_location_id', 'beach_key', 'hst', 'tpt', 'dpt',
       'Inshore_Latitude', 'Inshore_longitude', 'Last_date', 'beach',
       'distance', 'date_y'],
      dtype='object')

In [51]:
target = "stings_binary"
features = [ 'ts_wind', 'uas_wind','vas_wind', 
             'temp',
            'is_weekend','lat','lon','embaymentisation', 'orientation', 'day_of_year',] 

In [52]:
print(df_combined.shape)
df_combined.dropna(subset=features + [target], inplace=True)
df_combined.reset_index(drop=True, inplace=True)
print(df_combined.shape)

(37495, 44)
(37495, 44)


In [ ]:
# scaler = MinMaxScaler()
# df3[features] = scaler.fit_transform(df3[features])

In [53]:
df_combined['stings_binary'].value_counts()

stings_binary
1    31305
0     6190
Name: count, dtype: int64

### 1-Undersample

In [54]:
# Define your training and testing years
train_years = [ 2010, 2011,2012,2012,2013,2014,2015,2016,2017,2018,2019]
test_years = [2020,2021,2022]

# Split the data
train_df = df_combined[df_combined['year'].isin(train_years)]
test_df = df_combined[df_combined['year'].isin(test_years)]

# Create X and y sets
X_train = train_df[features]
y_train = train_df[target]

X_test = test_df[features]
y_test = test_df[target]


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=50)



#OverSample
ros = RandomOverSampler(random_state=42)
X_train_res, y_train_res = ros.fit_resample(X_train, y_train)



#Model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train_res, y_train_res)

# Eval
# y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.60).astype(int)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# y_test=df2_filtered[target]

# y_prob = model.predict_proba(df2_filtered[features])[:, 1]
# y_pred = (y_prob >= 0.60).astype(int)
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# y_test=df3['stings_binary2']

# y_prob = model.predict_proba(df3[features])[:, 1]
# y_pred = (y_prob >= 0.60).astype(int)
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)

# Plot it
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# import shap


# explainer = shap.Explainer(model, X_train)
# shap_values = explainer(X_train)


# shap.summary_plot(shap_values, X_train, plot_type="bar")
# shap.summary_plot(shap_values, X_train)

In [ ]:
df_subset=df3.loc[X_test.index,:]

In [ ]:
df_subset.reset_index(drop=True, inplace=True)

In [ ]:
df_subset['y_test'] = y_test.reset_index(drop=True)

In [ ]:
df_subset['y_pred'] = y_pred 

In [ ]:
df_subset['y_pred'] = df_subset['y_pred'].astype(int)
df_subset['y_test'] = df_subset['y_test'].astype(int)


In [ ]:
df_subset

In [ ]:
df_subset.to_csv('/home/mandana/Data/Pred3.csv', index=False)

In [ ]:
df_subset[(df_subset['y_pred'] == 0) & (df_subset['y_test'] == 1) ]['is_weekend'].value_counts()

In [ ]:
df_subset[(df_subset['y_pred'] == 0) & (df_subset['y_test'] == 1) ]['beach_key_lower'].value_counts()

In [ ]:
df_subset[(df_subset['y_pred'] == 0) & (df_subset['y_test'] == 1) ]['stings'].value_counts()

In [ ]:
df_merged2= pd.merge(df_subset, df3[[ 'beach_key_lower', 'lat','lon']], on=['beach_key_lower'], how='left')

In [ ]:
df_merged2.to_csv('/home/mandana/Data/Jan_stings_test1_pred_merged.csv', index=False)